<a href="https://colab.research.google.com/github/TheNizzo/SentimentAnalysis/blob/main/NLP_TP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
import random as rdn
import re

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset_train = load_dataset('imdb', split='train')
dataset_test = load_dataset('imdb', split='test')

In [72]:
x_train, y_train, x_test, y_test = dataset_train[:]['text'], dataset_train[:]['label'], dataset_test[:]['text'], dataset_test[:]['label']
len(x_train), len(x_test)

(25000, 25000)

In [77]:
0 in y_train

True

#pretreatment

* Take out html

In [4]:
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

In [5]:
x_train_pretreated = [remove_tags(el) for el in x_train]
x_test_pretreated = [remove_tags(el) for el in x_test]

# Training

In [ ]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip
%cd fastText-0.9.2
!make
!pip install .

In [7]:
import fasttext

In [8]:
def write_list_to_file(filename, list_to_write):
  textfile = open(filename, "w")
  for element in list_to_write:
      textfile.write(element + "\n")
  textfile.close()

In [9]:
def fasttext_conversion(list_y, list_x):
  return ["__label__" + str(y) + " " + x for y, x in zip(list_y, list_x)]

In [89]:
def predict_many(x, y, n, predict_model, verbose=False):
  for i in range(n):
    r = rdn.randint(0,25000)
    print("index = ", r)
    print(predict_model.predict(x[r]), "| Actual value :", y[r])
    if verbose :
      print(x[r])

* No pretreament

In [10]:
train = fasttext_conversion(y_train,x_train)
test = fasttext_conversion(y_test,x_test)

In [11]:
rdn.shuffle(train)
rdn.shuffle(test)

In [12]:
write_list_to_file("train_file.txt", train)
write_list_to_file("test_file.txt", test)

In [13]:
model = fasttext.train_supervised(input="train_file.txt")

In [90]:
predict_many(x_test, y_test, 10, model)

index =  19341
(('__label__0',), array([0.89549989])) | Actual value : 0
index =  14519
(('__label__1',), array([0.74022061])) | Actual value : 0
index =  11219
(('__label__1',), array([0.95898438])) | Actual value : 1
index =  23327
(('__label__0',), array([0.85880053])) | Actual value : 0
index =  14064
(('__label__0',), array([0.5261578])) | Actual value : 0
index =  3794
(('__label__1',), array([0.61225998])) | Actual value : 1
index =  12886
(('__label__0',), array([0.93984586])) | Actual value : 0
index =  15462
(('__label__0',), array([0.81752747])) | Actual value : 0
index =  19172
(('__label__0',), array([0.99708688])) | Actual value : 0
index =  6036
(('__label__1',), array([0.90136617])) | Actual value : 1


In [14]:
model.test("test_file.txt")

(25000, 0.85972, 0.85972)

* With pretreatment

In [15]:
train_pretreated = fasttext_conversion(y_train, x_train_pretreated)
test_pretreated = fasttext_conversion(y_test, x_test_pretreated)

In [16]:
rdn.shuffle(train_pretreated)
rdn.shuffle(test_pretreated)

In [17]:
write_list_to_file("train_file_pretreated.txt", train_pretreated)
write_list_to_file("test_file_pretreated.txt", test_pretreated)

In [84]:
model_pretreated = fasttext.train_supervised(input="train_file_pretreated.txt")

In [91]:
predict_many(x_test_pretreated, y_test, 10, model_pretreated, True)

index =  5641
(('__label__1',), array([0.66087395])) | Actual value : 1
I just thought I would add another observation, here. While there are a couple of visual sub-references, in this episode, to the possibility of unexpressed feelings between Jim Kirk and Yeoman Janice Rand; there is a special, physically tender, moment, toward the end. When the Enterprise is reversing at emergency warp speed in an attempt to outrun a possibly fatal Romulan plasma ball, Janice, perhaps fearing that their life is about to come to a dramatic end, seeks comfort by placing her head on Jim's sympathetic shoulder as they observe the aproaching instrument of their impending doom on the main viewer. I thought it was sweet (and Janice, of course, is gorgeous!).(P.S. Goof:- Several times, while supposedly firing phasors, the film shows photon torpedoes being launched)
index =  857
(('__label__1',), array([0.81803107])) | Actual value : 1
When it comes to movies, I don't easily discriminate between crap, pure c

In [88]:
model_pretreated.test("test_file_pretreated.txt")

(25000, 0.86072, 0.86072)